In [1]:
# Importation des bibliothèques

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import hamming_loss
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.dummy import DummyClassifier
import joblib
import pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Extraction des données

df = pd.read_csv("clean_dataset.csv")
df.head(3)

,Tags,Text
0,['c#'],error text textobject textobject script canvas...
1,['android-emulator'],android macos big sur mac big sur none working...
2,"['python', 'beautifulsoup']",invalidurl url control character request urlli...


In [3]:
# X: Variables prédictives et Y: Variables à prédire

X = df["Text"]
Y = df["Tags"]

In [4]:
# transformations des variables prédictives et à prédire en matrice de nombres binaires


vectorizer = TfidfVectorizer(analyzer = 'word',
                     min_df=0.0,
                     max_df = 1.0,
                     strip_accents = None,
                     encoding = 'utf-8', 
                     preprocessor=None,
                     token_pattern=r"(?u)\S\S+",
                     max_features=1000)
vectorizer.fit(X)
X1 = vectorizer.transform(X)

mlb = MultiLabelBinarizer()
Y1 = mlb.fit_transform(Y)

X1.shape,Y1.shape

((893, 1000), (893, 42))

In [5]:
file_ = open("vecto.pkl", "wb")
pickle.dump(vectorizer, file_)
file_.close()

In [6]:
file_ = open("vecto.pkl", "rb")
tfid1 = pickle.load(file_)

In [7]:
# données d'entraînement et données de test 

X_train, X_test, Y_train, Y_test = train_test_split(X1, Y1, test_size = 0.20, random_state = 42)

In [8]:
# testons prémièrement notre jeu de données avec l'algorithme DummyClassifier afin d'avoir une référence

clf_dummy = DummyClassifier(random_state=42) 
clf_dummy.fit(X_train, Y_train)
y_pred1 = clf_dummy.predict(X_test)
print("La fraction d'étiquettes qui sont mal prédites est de", round(hamming_loss(Y_test, y_pred1),3))

La fraction d'étiquettes qui sont mal prédites est de 0.275


In [9]:
# Mise en place de l'algorithme LogisticRegression via OneVsRest

model = LogisticRegression()
ovr = OneVsRestClassifier(model)
ovr.fit(X_train, Y_train)
y_pred2 = ovr.predict(X_test)
print("La fraction d'étiquettes qui sont mal prédites est de", round(hamming_loss(Y_test, y_pred2),3))

La fraction d'étiquettes qui sont mal prédites est de 0.151


In [10]:
# Adjustement des hyper-paramètres en vue d'améliorer la précision de l'algorithme

tuned_parameters = [{'estimator__C': [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]}]

hamming = make_scorer(hamming_loss)

log_reg_clf = OneVsRestClassifier(LogisticRegression())
logistic_gs = GridSearchCV(log_reg_clf, tuned_parameters,scoring = hamming, cv=3)
logistic_gs.fit(X_train, Y_train)
results = pd.DataFrame(logistic_gs.cv_results_)

results.sort_values(by = ["rank_test_score"], ascending = False).head(1)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator__C,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,0.492518,0.004139,0.00417,0.000045,10,{'estimator__C': 10},0.153061,0.147259,0.15076,0.15036,0.002386,9


In [11]:
# Modèle optimisé

model = LogisticRegression(C = 10)
ovr = OneVsRestClassifier(model)
ovr.fit(X_train, Y_train)
y_pred2 = ovr.predict(X_test)
print("La fraction d'étiquettes qui sont mal prédites est de", round(hamming_loss(Y_test, y_pred2),3))

La fraction d'étiquettes qui sont mal prédites est de 0.135


In [12]:
# Sauvegarde du modèle d'entraînement
file = open("model.pkl","wb")
pickle.dump(ovr, file)
file.close()